In [1]:
import pandas as pd
import os
from ast import literal_eval
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from typing import List
import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from nltk.stem import PorterStemmer  
porter_stemmer = PorterStemmer()

stop_words = set(stopwords.words())

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
import community.community_louvain as community
from ast import literal_eval
import re
from collections import defaultdict, Counter

from utils import *

[nltk_data] Downloading package stopwords to /home/selim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/selim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/selim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/selim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: cannot import name 'DatasetInfo' from 'huggingface_hub.hf_api' (/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/huggingface_hub/hf_api.py)

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

DATA_PATH = os.path.join(
    '..', '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))

/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
full_df

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,affected_groups_level_3,age,gender,source_type,url,website,lang,translation_en,translation_fr,translation_es
0,16851,"During the reporting week, IOM provided medica...",137.0,6334.0,322.0,False,['Health'],['Capacities & Response->International Response'],[],['Yemen'],...,[],[],[],website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int,en,NaN,"Au cours de la semaine de reporting, l'OIM a f...","Durante la semana de presentación de informes,..."
1,489433,Primary and secondary net enrollment rates are...,1306.0,67488.0,2225.0,False,['Education'],['Humanitarian Conditions->Living Standards'],['Context->Socio Cultural'],['République démocratique du Congo'],...,[],['Children/Youth (5 to 17 years old)'],[],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org,en,NaN,Les taux de scolarisation nets primaires et se...,Las tasas de inscripción netas primarias y sec...
2,186155,Impacto en las personas a. Inseguridad aliment...,829.0,41125.0,1898.0,False,"['Nutrition', 'Food Security']","['Impact->Number Of People Affected', 'At Risk...",['Shock/Event->Underlying/Aggravating Factors'...,['El Salvador'],...,[],['Infants/Toddlers (<5 years old)'],[],website,https://www.acaps.org/sites/acaps/files/key-do...,https://www.acaps.org,es,Impact on people a. Food insecurity Recurrent ...,Impact sur les gens a. Insécurité alimentaire ...,NaN
3,187037,"• According to the World Bank, most Venezuelan...",829.0,41081.0,1184.0,False,[],[],"['Displacement->Type/Numbers/Movements', 'Huma...",['Ecuador'],...,"['NOT_MAPPED', 'Migrants', 'None']",[],[],website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int,en,NaN,"• Selon la Banque mondiale, la plupart des mig...","• Según el Banco Mundial, la mayoría de los mi..."
4,489430,"Like few other countries globally, the majorit...",1306.0,67488.0,2225.0,False,[],[],['Context->Socio Cultural'],['République démocratique du Congo'],...,[],['Children/Youth (5 to 17 years old)'],[],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org,en,NaN,"Comme peu d'autres pays à l'échelle mondiale, ...",Al igual que algunos otros países a nivel mund...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157943,498755,"With the arrival of the pandemic, the voluntee...",829.0,41882.0,1187.0,NaN,[],[],[],[],...,[],[],[],website,https://redhum.org/documento/3680634,redhum.org,en,NaN,"Avec l'arrivée de la pandémie, les volontaires...","Con la llegada de la pandemia, los voluntarios..."
157944,498754,"With the arrival of the pandemic, the voluntee...",829.0,41882.0,1187.0,NaN,[],[],[],[],...,[],[],[],website,https://redhum.org/documento/3680634,redhum.org,en,NaN,"Avec l'arrivée de la pandémie, les volontaires...","Con la llegada de la pandemia, los voluntarios..."
157945,500721,"In 2021, UNICEF has prioritized actions in Bol...",829.0,60143.0,2074.0,NaN,[],[],[],[],...,[],[],[],website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int,en,NaN,"En 2021, l'UNICEF a hiérarchisé les actions en...","En 2021, UNICEF tiene acciones priorizadas en ..."
157946,500108,El 29 por ciento de las mujeres y el 24 por ci...,829.0,60058.0,1232.0,NaN,[],[],[],[],...,['None'],"['Adult (18 to 59 years old)', 'Adult (18 to 5...","['Female', 'Female']",website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int,es,29 percent of women and 24 percent of men who ...,29% des femmes et 24% des hommes qui ont accès...,NaN


In [10]:
def custom_eval(x):
    if str(x)=='nan':
        return []
    if str(x)=='[None]':
        return []
    if type(x)==list:
        return x
    else:
        return literal_eval(x)

kept_cols = ['sectors', 'affected_groups_level_3', 'specific_needs_groups', 'age', 'gender', 'subpillars_1d', 'subpillars_2d', 'severity', 'geo_location']
for col in kept_cols:
    full_df[col] = full_df[col].apply(custom_eval)

In [30]:
all_cols = ['entry_id', 'excerpt', 'project_id'] + kept_cols 
full_df[full_df.project_id.isin([1621, 2225, 2311])][all_cols].to_csv('summarization_df.csv', index=None)

In [ ]:
full_df['pillars_2d'] = full_df.subpillars_2d.apply(lambda x: [item.split('->')[0] for item in x])
full_df['pillars_1d'] = full_df.subpillars_1d.apply(lambda x: [item.split('->')[0] for item in x])
full_df['affected_groups'] = full_df.affected_groups_level_3.apply(
    lambda x: [item for item in x if item not in ['None', 'Others of Concern']]
)
full_df['sectors'] = full_df.sectors.apply(
    lambda x: [item for item in x if item not in ['Cross']]
)
full_df['n_sectors'] = full_df.sectors.apply(
    lambda x: len(x)
)
full_df['severity_scores'] = full_df['severity'].apply(get_severity_score)

#Counter(flatten(libya_df.severity)).most_common(5)

In [4]:
"""project_id_to_name = {
    1621: 'DFS Libya',
    2225: 'IMMAP/DFS RDC',
    2311: 'IMMAP/DFS Colombia'
}

def preprocess_entry(entry):
    def remove_punct(sentence):
        import string
        return sentence.translate(str.maketrans('', '', string.punctuation))

    def omit_stop_words(sentence):
        return ' '.join([word for word in sentence.split() if word.lower() not in stop_words])

    return omit_stop_words(remove_punct(entry))"""

In [5]:
"""between_sequence_links = [
    'Furthermore, ', 'Aditionally, ', 'Moreover, ', 'Besides, ', 'Aside from that, ', 'Also, ', 'In addition to that, ',
    'On the other hand, ', 'On the other side, '
    ]
final_sent_link = ['Finally, ', 'Ultimately, ']
first_sent_link = ['Firstly, ', 'First of all, ', 'In the first place, ']"""

def get_summary_one_row(row):

    """
    function used for getting summary for one task
    """
    final_summary = []

    original_excerpt = row.excerpt
    severity_scores = row.severity_scores
    top_n_sentences = row.n_clusters

    cosine_similarity_matrix = get_similarity_matrix(original_excerpt)

    graph_one_lang = build_graph(cosine_similarity_matrix)

    scores_pagerank = nx.pagerank(graph_one_lang)
    scores = {key: value * severity_scores[key] for key, value in scores_pagerank.items()}

    top_n_sentence_ids = np.argsort(np.array(list(scores.values())))[::-1][:top_n_sentences]
    used_ids = []

    for id_tmp in top_n_sentence_ids:
        row_id = cosine_similarity_matrix[id_tmp, :]
        top_id_row = np.argsort(row_id)[::-1]
        top_id_row = [id for id in top_id_row if id not in used_ids and id not in top_n_sentence_ids][:1]

        top_2_id_row = [id_tmp] + top_id_row

        used_ids += top_2_id_row

        ranked_sentence = ' '.join([original_excerpt[id_tmp] for id_tmp in (top_2_id_row)]) 
            
        summarized_entries = t2t_generation(ranked_sentence)
        capitalized_summaries = ' '.join([sent.capitalize() for sent in nltk.tokenize.sent_tokenize(summarized_entries)])
        final_summary.append(capitalized_summaries)

    final_summary_str = ' '.join(final_summary)
    
    return final_summary_str

def get_summary_one_part(df, col_name: str):

    print(f'BEGIN {col_name}')
    
    preprocessed_df = process_df(df, col_name)

    final_returns = {}
    for index, row in preprocessed_df.iterrows():
        tag_tmp = row.tmp_tag_str
        print(f'For: {tag_tmp}:')
        summarized_text = get_summary_one_row(row)
        print(summarized_text)
        final_returns[tag_tmp] = summarized_text

    return final_returns

def get_report_relation_other_sectors(df):
    many_tags_df = df[df.n_sectors>1].copy()
    many_tags_df['sectors'] = many_tags_df.sectors.apply(
        lambda x: [item for item in x if item!='Cross']
    )
    final_dict = {}

    """many_sectors_df = many_tags_df[many_tags_df.sectors.apply(lambda x: len(x)>2)]
    final_dict.update(get_summary_one_part(many_sectors_df, 'sectors'))"""

    two_sectors_df = many_tags_df[many_tags_df.sectors.apply(lambda x: len(x)==2)]
    return get_summary_one_part(two_sectors_df, 'sectors')

def get_report_relation_affected_pops(df):

    final_dict = {}
    for col_tmp in ['age', 'specific_needs_groups', 'affected_groups']:
        final_dict.update(get_summary_one_part(df, col_tmp))

    return final_dict

def get_report_hum_needs(df):
    hum_conds_df = df[df.pillars_2d.apply(lambda x: 'Humanitarian Conditions' in x)].copy()
    hum_conds_df['pillars_1d_no_casualties'] = hum_conds_df.pillars_1d.apply(
        lambda x: list(set([item for item in x if item!='Casualties']))
    )
    return get_summary_one_part(hum_conds_df, 'pillars_1d_no_casualties')

def get_report(full_df, tag: str, project_id: int):
    """
    main function to get full report
    """
    df = full_df[full_df.project_id==project_id].copy()
    df_one_sector = df[df.sectors.apply(lambda x: tag in x)]
    print(df_one_sector.shape)

    summaries = {}
    
    # secondary tags
    summaries['affected_specific'] = get_report_relation_affected_pops(df_one_sector)
    
    # key trends
    summaries['key_trends'] = get_report_hum_needs(df_one_sector)

    #relation to other sectors
    summaries['other_sectors'] = get_report_relation_other_sectors(df_one_sector)

    return summaries

In [6]:
full_report = get_report(full_df, 'Health', 1621)

(377, 27)


In [14]:
import json
with open('report.json', 'w') as fp:
    json.dump(full_report, fp)